# Работа с базами данных в Python

## DB API
DB API это стандартизированный набор методов для работы с базами данных в Python. 

![db-api](dbapi.png)

Предварительная подготовка: 
1. Запустить БД, если она еще не запущена. Будем использовать mssql в docker `docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=1234qwE?" -e "MSSQL_PID=Express" -p 1433:1433 -d mcr.microsoft.com/mssql/server:2022-latest`
2. Установить драйвер для mssql `uv add mssql-python`
3. Научимся подключаться из приложений для работы с бд (dbeaver, datagrip и тп)
4. Создать тестовую базу данных и наполнить ее данными

In [ ]:
!docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=1234qwE?" -e "MSSQL_PID=Express" -p 1433:1433 -d mcr.microsoft.com/mssql/server:2022-latest

In [12]:
!uv add mssql-python

Resolved 47 packages in 15ms
Audited 45 packages in 9ms


# Установка соединения

In [1]:
# Просто подключимся к БД


## Что тут происходит
1. Создали connection string
2. Открыли подключение, передав строку в `connect`
3. Создали курсор - объект для работы с БД
4. Затем, так как подключение и курсор это внешние по отношению к питону ресурсы мы должны их освободить с помощью методов `close`

## Как устроен connection string
Строка это набор элементов `<key>=<value>` описывающая то как надо подключаться к БД

|key|Описание|
|:--|:-------|
|Server|Адрес подключения к бд|
|Port|Порт подключения к бд|
|Database|Имя самой базы данных|
|UID|Имя пользователя|
|PWD|Пароль пользователя|
|Encrypt|Включение/выключение шифрования отправляемых данных|

Кроме этих полей есть еще несколько других о которых можно прочитать в [документации](https://github.com/microsoft/mssql-python/wiki/Connection-to-SQL-Database#connection-string) к драйверу.

# Чтение данных из БД

In [2]:
# Читаем из БД


Выгружать из базы все не всегда удобно (хотя курсор все равно выгржает все данные в память), поэтому иногда имеет смысл читать ответ построчно

In [3]:
# Читаем из БД построчно


In [4]:
# Курсор как итератор


# Запись данных в БД

In [5]:
# Сперва пишем, а потом читаем


Часто бывает необходимо выполнять параметризованные запросы: пользователь передает в вашу функцию название айтема и флаг выполнен ли он, а вы должны вставить. Ваши предложения, как это можно сделать?

In [6]:
# Передача параметризованных запросов: именованные и не именованные параметры


Что делать если хотим вставить произвольное количество строк в таблицу? 

In [7]:
# Передача множества параметров


# Транзакции

Транзакция - это последовательность операций, которая может выполниться полностью, либо не выполниться вообще. При создании подключения и курсора создается новая транзакция в рамках которой выполняются операции над БД. После этого транзакцию можно либо применить (`commit`), либо откатить (`rollback`). Если закрыть подключение без применения транзакции, то все выполненные изменения откатятся.

In [8]:
# Напишем код с коммитом транзакции


In [9]:
import mssql_python

connection_string = 'SERVER=localhost;PORT=1433;DATABASE=todo;UID=sa;PWD=1234qwE?;Encrypt=no'
conn = mssql_python.connect(connection_string)
cur = conn.cursor()

cur.execute('SELECT * from items')

print(cur.fetchall())

cur.close()
conn.close()

[('buy bread', True), ('buy milk', False), ('watch tv', True), ('play videogames', True), ('make dinner', False), ('read book', True), ('watch tv', True), ('play videogames', True), ('watch tv', True), ('play videogames', True), ('watch tv', True), ('play videogames', True), ('make dinner', False), ('read book', True), ('watch tv', True), ('play videogames', True), ('watch tv', True), ('play videogames', True), ('watch tv', True), ('play videogames', True), ('make dinner', False)]


# Как не забывать закрыть подключение

In [10]:
# Перепишем код на менеджеры контекста with
